In [1]:
import json
import os
import autogen
from autogen import ConversableAgent

In [2]:
llm_config = {"model": "llama3.1","api_type":"ollama"}

In [3]:
Information_agent = ConversableAgent(
    name="Information_agent",
    system_message='''You are a helpful Travel agent.
    You are here to help new customers get their Travel planned.
    Your job is to gather customer's destination, travel days, number of people and Budget.
    (e.g., {
  "destination": "Kodaikanal",
  "Travel Days": "4 days",
  "number of people": "6"
  "Budget": "INR 40000"})
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

Budget_Agent = ConversableAgent(
    name="Budget_Agent",
    system_message='''You are a helpful customer Budget_Agent.
    You are here to gather information from all the agents and create a plan based on budget.
    you calculate estimated costs. All should be
    (e.g., calculate hotel stay price, price of restaurant to eat, price for place to visit and activities).
    All we mentioned in example should come under the customer budget.
    Suggests changes if the plan exceeds budget.
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

Sight_seeing_Agent = ConversableAgent(
    name="SightseeingAgent",
    system_message='''You are a helpful customer SightseeingAgent agent.
    You are here to help new customers get best sight seeing (Tourist spots) in their preferred destination.
    Your job is to gather customer's interests for sight seeing. Sight they want to see and Activities they want to do.
    (e.g., {'Sight':'Nature', 'Activity':'Boat Riding'}).
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

Food_Agent = ConversableAgent(
    name="Food_Agent",
    system_message='''You are a helpful customer Food agent.
    You are here to help new customers get best restaurants, hotels of their preferred cuisine.
    Your job is to gather customer's food preferences (e.g., non-veg) cuisine type (e.g., local cuisine, continental).
    Ensure restaurants are near sightseeing spots.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

Travel_agent = ConversableAgent(
    name="Travel_agent",
    system_message='''You are a helpful customer Travel_agent.
    You aggregate the information gathered by other agents.
    Your job is to generate a list of 3 search terms related to the information gathered by other agents.
    For each search term, `search_google` and analyze the results.
    From the results of all searches, return the 3 most relevant results to the user's preferences.
    Remember: the quality of the results is important.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [4]:
chats = [
    {
        "sender": Travel_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our service. Could you tell me your Starting location, destination, Travel days, number of people?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'destination': '', 'Travel Days': '', 'number of people': '', 'Budget':''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": Sight_seeing_Agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me your preferred sights and activities you like to do?",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'sight':'', 'activities':''}",
        },
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
    {
        "sender": Food_Agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me your food preferences?",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'food preferences':'', 'cuisine type':''}",
        },
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
    {
        "sender": customer_proxy_agent,
        "recipient": Budget_Agent,
        "message": "Aggregrate information from all agents and give final suggestions to the customer on Budget.",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": customer_proxy_agent,
        "recipient": Travel_agent,
        "message": "Aggregrate information from all agents and give final suggestions to the customer.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "clear_history": True
    },
]


In [5]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Travel_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our service. Could you tell me your Starting location, destination, Travel days, number of people?

--------------------------------------------------------------------------------


C:\Anaconda\envs\AgenticAI\lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Travel_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Wayanad, 3 days, 3 people, INR 50000


customer_proxy_agent (to Travel_agent):

Wayanad, 3 days, 3 people, INR 50000

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (8b395e3a-6da1-43df-8e61-facb4f97a4b6): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
SightseeingAgent (to customer_proxy_agent):

Great! Could you tell me your preferred sights and activities you like to do?
Context: 
{
  "destination": "Wayanad",
  "Travel Days": "3 days",
  "number of people": "3 people",
  "Budget": "INR 50000"
}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to SightseeingAgent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Lakes, Horse Riding


customer_proxy_agent (to SightseeingAgent):

Lakes, Horse Riding

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (89f7311a-7314-4405-9ee6-a3bc9b2dd05d): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
Food_Agent (to customer_proxy_agent):

Great! Could you tell me your food preferences?
Context: 
{
  "destination": "Wayanad",
  "Travel Days": "3 days",
  "number of people": "3 people",
  "Budget": "INR 50000"
}
{
  "sight": "Lakes",
  "activities": "Horse Riding"
}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Food_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Non-vegetarian, local cuisine


customer_proxy_agent (to Food_Agent):

Non-vegetarian, local cuisine

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (893cc3bd-e92f-4e4c-98ec-64789d5cdb27): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Budget_Agent):

Aggregrate information from all agents and give final suggestions to the customer on Budget.
Context: 
{
  "destination": "Wayanad",
  "Travel Days": "3 days",
  "number of people": "3 people",
  "Budget": "INR 50000"
}
{
  "sight": "Lakes",
  "activities": "Horse Riding"
}
{'food preferences': 'non-vegetarian', 'cuisine type': 'local'}

--------------------------------------------------------------------------------
Budget_Agent (to customer_proxy_agent):

After gathering information from all the agents, I have calculated